In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!nvidia-smi

Fri Feb 25 17:00:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install kaggle

!mkdir -p ~/.kaggle                                               
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d rowhitswami/nips-papers-1987-2019-updated
!unzip /content/nips-papers-1987-2019-updated.zip

!kaggle datasets download -d littleotter/united-states-presidential-speeches
!unzip /content/united-states-presidential-speeches.zip

!kaggle datasets download -d unitednations/un-general-debates
!unzip /content/un-general-debates.zip

 96% 102M/106M [00:00<00:00, 197MB/s] 
100% 106M/106M [00:00<00:00, 240MB/s]
Archive:  /content/nips-papers-1987-2019-updated.zip
  inflating: authors.csv             
  inflating: papers.csv              
 97% 21.0M/21.7M [00:00<00:00, 106MB/s]
100% 21.7M/21.7M [00:00<00:00, 107MB/s]
Archive:  /content/united-states-presidential-speeches.zip
  inflating: corpus.csv              
  inflating: fifth_party_corpus.csv  
  inflating: first_party_corpus.csv  
  inflating: fourth_party_corpus.csv  
  inflating: presidential_speeches.csv  
  inflating: second_party_corpus.csv  
  inflating: sixth_party_corpus.csv  
  inflating: third_party_corpus.csv  
 96% 42.0M/43.6M [00:00<00:00, 151MB/s]
100% 43.6M/43.6M [00:00<00:00, 214MB/s]
Archive:  /content/un-general-debates.zip
  inflating: un-general-debates.csv  


In [ ]:
#!CUDA_LAUNCH_BLOCKING=1
!pip install gpytorch torch --upgrade

     |████████████████████████████████| 310 kB 8.1 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:41tcmalloc: large alloc 1147494400 bytes == 0x562a20f4c000 @  0x7f89e80ff615 0x562a1eba23bc 0x562a1ec8318a 0x562a1eba51cd 0x562a1ec97b3d 0x562a1ec19458 0x562a1ec1402f 0x562a1eba6aba 0x562a1ec192c0 0x562a1ec1402f 0x562a1eba6aba 0x562a1ec15cd4 0x562a1ec98986 0x562a1ec15350 0x562a1ec98986 0x562a1ec15350 0x562a1ec98986 0x562a1ec15350 0x562a1eba6f19 0x562a1ebeaa79 0x562a1eba5b32 0x562a1ec191dd 0x562a1ec1402f 0x562a1eba6aba 0x562a1ec15cd4 0x562a1ec1402f 0x562a1eba6aba 0x562a1ec14eae 0x562a1eba69da 0x562a1ec15108 0x562a1ec1402f
     |████████████████████████████████| 881.9 MB 1.8 kB/s 
  Created wheel for gpytorch: filename=gpytorch-1.6.0-py2.py3-none-any.whl size=509889 sha256=85f89531f09dce23790c34341cc32296d2ff392754f99b043f46ead11476fcb6
  Stored in directory: /root/.cache/pip/wheels/66/b5/89/34c06ad393a6feb72b4cdde46d0f1c667f3e2632960f9df109
Successfully built gpyto

In [ ]:
from gpytorch.models.gplvm.latent_variable import *
from gpytorch.models.gplvm.bayesian_gplvm import BayesianGPLVM
from matplotlib import pyplot as plt
from tqdm.notebook import trange
from gpytorch.means import ZeroMean
from gpytorch.mlls import VariationalELBO
from gpytorch.priors import NormalPrior
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.variational import VariationalStrategy
from gpytorch.variational import CholeskyVariationalDistribution,NaturalVariationalDistribution, TrilNaturalVariationalDistribution
from gpytorch.kernels import ScaleKernel, RBFKernel,MaternKernel
from gpytorch.distributions import MultivariateNormal

Model

In [ ]:
import gpytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import dtype, Tensor
from torch.distributions.constraints import positive
from torch.nn.modules.module import T
from torch.utils.data import Dataset
from typing import Optional, Union, overload

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class Encoder(nn.Module):
    def __init__(self, vocab_size, num_topics, hidden, dropout, batchNorm):
        super().__init__()
        if torch.cuda.is_available():
            self.cuda()

        self.num_topics = num_topics
        self.batchNorm = batchNorm
        self.drop = nn.Dropout(dropout)  # dropout
        self.fc1 = nn.Linear(vocab_size+num_topics, hidden)
        self.fc2 = nn.Linear(hidden+num_topics, hidden)
        self.fcmu = nn.Linear(hidden+num_topics, num_topics, bias=True)  # fully-connected layer output mu
        self.fclv = nn.Linear(hidden+num_topics, num_topics, bias=True)  # fully-connected layer output sigma
        self.act1 = nn.Softplus()
        self.act2 = nn.Softplus()
        
        self.norm1 = nn.LayerNorm(hidden, elementwise_affine=False)
        self.norm2 = nn.LayerNorm(hidden, elementwise_affine=False)
        
        self.bnmu = nn.BatchNorm1d(num_topics, affine=False)  # to avoid component collapse
        self.bnlv = nn.BatchNorm1d(num_topics, affine=False)  # to avoid component collapse

        
    def get_kl(self, q_mu, q_logsigma, p_mu=None, p_logsigma=None):
        """ Gaussian KL Divergence
        Returns KL( N(q_mu, q_logsigma) || N(p_mu, p_logsigma) ).
        """
        if p_mu is not None and p_logsigma is not None:
            sigma_q_sq = torch.exp(q_logsigma).to(device)
            sigma_p_sq = torch.exp(p_logsigma).to(device)
            kl = (sigma_q_sq + (q_mu - p_mu) ** 2) / (sigma_p_sq + 1e-6)
            kl = kl - 1 + p_logsigma - q_logsigma
            kl = 0.5 * torch.sum(kl, dim=-1).to(device)
        else:
            kl = -0.5 * torch.sum(1 + q_logsigma - q_mu.pow(2) - q_logsigma.exp(), dim=-1).to(device)
        return kl

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar).to(device)
        eps = torch.randn_like(std).to(device)
        return eps.mul_(std).add_(mu)

    def forward(self, inp, res_inp, eva=False):
        inp = torch.cat([inp, res_inp], dim=1).to(device)
        h = self.act1(self.fc1(inp))
        h = self.norm1(h) # layernorm
        inp = torch.cat([h, res_inp], dim=1).to(device)
        h = self.act2(self.fc2(inp))
        h = self.norm2(h) # layernorm
        h = self.drop(h)
        h = torch.cat([h, res_inp], dim=1).to(device)
        # μ and Σ are two inference networks
        mu_theta = self.fcmu(h)
        sig_theta = self.fclv(h)
        if eva:
            return mu_theta.softmax(-1)
        z = self.reparameterize(mu_theta, sig_theta)
        theta = torch.softmax(z, -1).to(device)
        kld_theta = self.get_kl(mu_theta, sig_theta, res_inp, 0.005*torch.randn(self.num_topics).to(device))
        return theta, kld_theta


class Decoder(nn.Module):
    # Pre-trained embedding, alpha, rho, embedding method
    # Need to be refactored with Topic Embedding
    def __init__(self, vocab_size, num_topics, num_times, dropout,
                 useEmbedding=True, rho_size=256, pre_embedding=None, emb_type='NN',
                 trainEmbedding=True):
        super().__init__()
        # this beta can be refactorized in to BOW, a neural network that to be trained
        self.emb_type = emb_type
        self.trainEmbedding = trainEmbedding
        self.useEmbedding = useEmbedding
        self.rho_size = rho_size
        # Changes - Beta
        # < Linear NN (K->V)
        # > Embedding -> Linear NN (K->E->V)
        self.drop = nn.Dropout(dropout)
        if self.useEmbedding:
            # Call ρ Topic Embedding

            if trainEmbedding:
                self.fcrho = TopicEmbedding(rho_size, vocab_size, pre_embedding,
                                            emb_type, dropout)
                self.bnrho = nn.BatchNorm1d(rho_size, affine=False)
            # use original embedding
            else:
                self.fcrho = TopicEmbedding(rho_size, vocab_size, pre_embedding,
                                            emb_type, dropout, trans_layer, trans_head, trans_dim)
            # Call α
            self.fcalpha = nn.Linear(rho_size, num_topics, bias=False)
            self.bnalpha = nn.BatchNorm1d(num_topics, affine=False)
            # nn.Parameter(torch.randn(rho_size, num_topics))
        else:
            # Call β, Use Original NN (K->V)
            self.fcbeta = nn.Parameter(torch.randn(num_times, num_topics, vocab_size).to(device))
            #self.bnbeta = nn.BatchNorm1d(vocab_size, affine=False)

        self.bn = nn.BatchNorm1d(vocab_size, affine=False)

        if torch.cuda.is_available():
            self.cuda()
            
    def get_beta(self, alpha):
        if self.trainEmbedding:
            if self.emb_type is 'NN':
#                 logit = self.fcrho(alpha.reshape(alpha.size(0) * alpha.size(1), self.rho_size))
#                 logit = logit.reshape(alpha.size(0), alpha.size(1), -1)
#                 beta = F.softmax(logit, dim=-1)
                n_alpha = alpha.reshape(alpha.size(0) * alpha.size(1), self.rho_size)
                rho = self.fcrho.rho.weight.T
                logit = torch.mm(n_alpha, rho)
                logit = logit.reshape(alpha.size(0), alpha.size(1), -1)
                beta = F.softmax(logit, dim=-1)
            else:
                raise ValueError('Wrong embedding type')
        elif self.useEmbedding:
            beta = self.bnalpha(self.fcalpha(self.bnrho(self.fcrho.weight()))).transpose(1, 0).to(device)
        else:
            beta = self.fcbeta.weight
        return beta


class TopicEmbedding(nn.Module):
    def __init__(self, rho_size, vocab_size, pre_embedding=None,
                 emb_type='NN', dropout=0.0,
                 n_heads=8, n_layer=4, n_dim=128, n_code=8):
        super().__init__()
        self.emb_type = emb_type
        if pre_embedding is None:
            # 1. Embedding layer
            if emb_type is 'NN':
                self.rho = nn.Linear(rho_size, vocab_size, bias=False)
            else:
                raise ValueError('Wrong Embedding Type')
        else:
            self.rho = pre_embedding.clone().float().to(device)
            
    def forward(self, inputs):
        if self.emb_type is 'NN':
            return self.rho(inputs)
        else:
            raise ValueError('Wrong Embedding Type')


class TETM(nn.Module):

    def __init__(self, vocab_size, num_topics, num_times, hidden, dropout,
                 delta, data_size, useEmbedding=False, eta_size=256, rho_size=256,
                 pre_embedding=None, emb_type='NN', trainEmbedding=False, batchNorm=True):
        super().__init__()

        self.delta = delta
        if torch.cuda.is_available():
            self.cuda()

        self.vocab_size = vocab_size
        self.num_topics = num_topics
        self.num_times = num_times
        self.useEmbedding = useEmbedding
        self.emb_type = emb_type
        self.rho_size = rho_size
        self.eta_size = eta_size

        self.encoder = Encoder(vocab_size, num_topics, hidden, dropout, batchNorm).to(device)
        self.decoder = Decoder(vocab_size, num_topics, num_times,dropout,
                               useEmbedding, rho_size, pre_embedding, emb_type,
                               trainEmbedding).to(device)
        ## define the variational parameters for the topic embeddings over time (alpha) ... alpha is K x T x L
        self.mu_q_alpha = nn.Parameter(torch.randn(num_topics, num_times, rho_size).to(device))
        self.logsigma_q_alpha = nn.Parameter(torch.randn(num_topics, num_times, rho_size).to(device))

        self.data_size = data_size
        self.gplvm = bGPLVM(self.data_size, vocab_size, self.num_topics, 100).to(device)
        self.likelihood = GaussianLikelihood(batch_shape=(num_times, vocab_size)).to(device)
        
        self.mu_q_eta = nn.Linear(self.num_topics, self.num_topics, bias=True).to(device)
        self.logsigma_q_eta = nn.Linear(self.num_topics, self.num_topics, bias=True).to(device)

        self.eta = nn.Parameter(torch.randn(self.num_times,self.num_topics).to(device))


    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar).to(device)
        eps = torch.randn_like(std).to(device)
        return eps.mul_(std).add_(mu)

    def get_beta(self,alpha):
        beta = self.decoder.get_beta(alpha)
        return beta
    
    def get_theta(self, eta, bows, times):
        eta_td = eta[times.type('torch.LongTensor')]
        theta, kl_theta = self.encoder(bows, eta_td)
        return theta, kl_theta

    def decode(self, theta, beta):
        res = torch.mm(theta, beta)
        preds = torch.log(res + 1e-6)
        return preds

    def get_kl(self, q_mu, q_logsigma, p_mu=None, p_logsigma=None):
        """ Gaussian KL Divergence
        Returns KL( N(q_mu, q_logsigma) || N(p_mu, p_logsigma) ).
        """
        if p_mu is not None and p_logsigma is not None:
            sigma_q_sq = torch.exp(q_logsigma).to(device)
            sigma_p_sq = torch.exp(p_logsigma).to(device)
            kl = (sigma_q_sq + (q_mu - p_mu) ** 2) / (sigma_p_sq + 1e-6)
            kl = kl - 1 + p_logsigma - q_logsigma
            kl = 0.5 * torch.sum(kl, dim=-1).to(device)
        else:
            kl = -0.5 * torch.sum(1 + q_logsigma - q_mu.pow(2) - q_logsigma.exp(), dim=-1).to(device)
        return kl
        
    def get_alpha(self):  ## mean field
        # TxKxL
        alphas = torch.zeros(self.num_times, self.num_topics, self.rho_size).to(device)
        kl_alpha = []
        alphas[0] = self.reparameterize(self.mu_q_alpha[:, 0, :], self.logsigma_q_alpha[:, 0, :])
        p_mu_0 = torch.zeros(self.num_topics, self.rho_size).to(device)
        logsigma_p_0 = torch.zeros(self.num_topics, self.rho_size).to(device)
        kl_0 = self.get_kl(self.mu_q_alpha[:, 0, :], self.logsigma_q_alpha[:, 0, :], p_mu_0, logsigma_p_0)
        kl_alpha.append(kl_0)
        for t in range(1, self.num_times):
            alphas[t] = self.reparameterize(self.mu_q_alpha[:, t, :], self.logsigma_q_alpha[:, t, :])
            p_mu_t = alphas[t - 1]
            logsigma_p_t = torch.log(self.delta * torch.ones(self.num_topics, self.rho_size).to(device))
            kl_t = self.get_kl(self.mu_q_alpha[:, t, :], self.logsigma_q_alpha[:, t, :], p_mu_t, logsigma_p_t)
            kl_alpha.append(kl_t)
        kl_alpha = torch.stack(kl_alpha).sum()
        return alphas, kl_alpha.sum()

    # Compute η[t]~N(η[t-1], δ^2*I), η[0]=
    def get_eta(self, eta_inp):
        etas = torch.zeros(self.num_times, self.num_topics).to(device)
        kl_eta = []
        inp_0 = eta_inp[0]#torch.cat([eta_inp[0], torch.zeros(self.num_topics, ).to(device)], dim=0).to(device)
        mu_0, logsigma_0 = self.mu_q_eta(inp_0), self.logsigma_q_eta(inp_0)
        etas[0] = self.reparameterize(mu_0, logsigma_0)
        p_mu_0, logsigma_p_0 = torch.zeros(self.num_topics, ).to(device), torch.zeros(self.num_topics, ).to(device)
        kl_0 = self.get_kl(mu_0, logsigma_0, p_mu_0, logsigma_p_0)
        kl_eta.append(kl_0)
        for t in range(1, self.num_times):
            inp_t = eta_inp[t]#torch.cat([eta_inp[t], etas[t - 1]], dim=0)
            mu_t, logsigma_t = self.mu_q_eta(inp_t), self.logsigma_q_eta(inp_t)
            etas[t] = self.reparameterize(mu_t, logsigma_t)
            logsigma_p_t = torch.log(self.delta * torch.ones(self.num_topics,).to(device))
            kl_t = self.get_kl(mu_t, logsigma_t, etas[t - 1], logsigma_p_t)
            kl_eta.append(kl_t)
        kl_eta = torch.stack(kl_eta).sum()
        return etas, kl_eta

    def init_hidden(self):
        """Initializes the first hidden state of the RNN used as inference network for \eta.
        """
        weight = next(self.parameters())
        nlayers = self.eta_nlayers
        nhid = self.eta_size
        return (weight.new_zeros(nlayers, 1, nhid), weight.new_zeros(nlayers, 1, nhid))
    
    def get_mu(self, rnn_inp):
        mll = VariationalELBO(self.likelihood, self.gplvm, num_data=len(rnn_inp))#,combine_terms=False)
        sample = self.gplvm.sample_latent_variable()
        output = self.gplvm(sample)
        loss = mll(output, rnn_inp.T.to(device))
        # nll + kl_x + kl_u
        return self.gplvm.X.q_mu, loss#loss[0].sum()+loss[1].sum()+loss[2].sum()+loss[3].sum()

    def forward(self, bows, norm_bows, times, rnn_inp, num_docs):
        bsz = bows.size(0)
        coeff = num_docs / bsz
        #eta, kl_eta = self.get_eta(rnn_inp)
        #eta_gp, kld_eta_gp = self.get_mu(rnn_inp)
        kld_eta = torch.zeros(()).to(device)
        #eta, kld_eta = self.get_eta(eta_gp.to(device))
        # get theta N(η,α^2I)
        theta, kld_theta = self.get_theta(self.eta, norm_bows, times)
        kld_theta = kld_theta.sum() * coeff
        alpha, kl_alpha = self.get_alpha()
        assert (alpha.shape == torch.Size(
            [self.num_times, self.num_topics, self.rho_size]
        ))
        beta = self.get_beta(alpha)
        beta = beta[times.type('torch.LongTensor')]

        assert (beta.shape == torch.Size(
            [bows.shape[0], self.num_topics, self.vocab_size]
        ))
        theta = theta.unsqueeze(1)
        assert (theta.shape == torch.Size([bows.shape[0], 1, self.num_topics]
                                        ))
        pred = torch.bmm(theta, beta).squeeze(1).nan_to_num()
        logp = torch.log(pred + 1e-6).nan_to_num()
        nll = -(logp * bows).sum(-1)
        nll = nll.sum() * coeff
        return nll, kl_alpha, kld_eta, kld_theta ,0#kld_eta_gp
    
    def get_beta_result(self):
        alpha = model.mu_q_alpha.clone().contiguous()
        alpha = alpha.permute(1,0,2)
        beta = model.get_beta(alpha, torch.arange(0,ts.unique().shape[0]))
        return beta
    
    def get_eta_result(self):
        return self.gplvm.X.q_mu

    def predict(self, d_bat, norm_d_bat, t_bat, rnn_inp):
        """give out the test data set, return the corresponding perplexity"""
        self.eval()
        with torch.no_grad():
            # get eta(TxK)
            #eta = self.gplvm.X.q_mu
            eta = self.eta
            #eta, kl_eta = self.get_eta(rnn_inp)
            assert (eta.shape == torch.Size([self.num_times, self.num_topics]))
            # get theta(DxK)
            eta_td = eta[t_bat.type('torch.LongTensor')]
            theta = self.encoder(d_bat, eta_td,eva=True)
            #theta = self.get_theta(eta, norm_d_bat, t_bat)[0]
            assert (theta.shape == torch.Size([norm_d_bat.shape[0], self.num_topics]))
            #theta = theta.unsqueeze(1)
            # get alpha(KxTxL)
            alpha = model.mu_q_alpha.clone().contiguous()
            #alpha = alpha.permute(1, 0, 2)
            assert (alpha.shape == torch.Size(
                [self.num_topics, self.num_times, self.rho_size]
            ))
            # alpha_td(KxDxL)
            #alpha_td = alpha[:,t_bat.type('torch.LongTensor'), :]
#             assert (alpha_td.shape == torch.Size(
#                 [self.num_topics, d_bat.shape[0], self.rho_size]
#             ))
            # get beta(T[D]xKxV)
            beta = self.get_beta(alpha)
            beta = beta.permute(1, 0, 2)
            beta = beta[t_bat.type('torch.LongTensor')]
            assert (beta.shape == torch.Size(
                [d_bat.shape[0], self.num_topics, self.vocab_size]
            ))
            loglik = theta.unsqueeze(2) * beta
            pred = loglik.sum(1)
            #pred = torch.bmm(theta, beta).squeeze(1)
            logp = torch.log(pred)
            sums = d_bat.sum(1).unsqueeze(1)
            loss = (-logp * d_bat).sum(-1) / sums.squeeze()
            loss = loss.nan_to_num().mean().item()
            # ppl check when doing mini-batch
            ppl = round(math.exp(loss), 1)
            return ppl


In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
class bGPLVM(BayesianGPLVM):
    def __init__(self, n, data_dim, latent_dim, n_inducing, pca=False, nu=2.5):
        self.n = n
        self.batch_shape = torch.Size([data_dim])
        self.inducing_inputs = torch.randn(data_dim, n_inducing, latent_dim).to(device)
        q_u = CholeskyVariationalDistribution(n_inducing)
        q_f = VariationalStrategy(self, self.inducing_inputs, q_u, learn_inducing_locations=True)
        X_prior_mean = torch.zeros(n, latent_dim).to(device)  # shape: N x Q
        prior_x = NormalPrior(X_prior_mean, torch.ones_like(X_prior_mean).to(device))
        X_init = torch.nn.Parameter(torch.randn(n, latent_dim))
        # LatentVariable (c)
        X = VariationalLatentVariable(n, data_dim, latent_dim, X_init, prior_x)
        super().__init__(X, q_f)
        self.mean_module = ZeroMean(ard_num_dims=latent_dim)
        self.covar_module = ScaleKernel(RBFKernel(ard_num_dims=latent_dim))
        init_lengthscale = 0.1
        self.covar_module.base_kernel.lengthscale = init_lengthscale

    def forward(self, X):
        mean_x = self.mean_module(X)
        covar_x = self.covar_module(X)
        dist = MultivariateNormal(mean_x, covar_x)
        return dist

    def _get_batch_idx(self, batch_size):
        valid_indices = np.arange(self.n)
        batch_indices = np.random.choice(valid_indices, size=batch_size, replace=False)
        return np.sort(batch_indices)


In [ ]:
from sklearn.manifold import TSNE
import torch 
import numpy as np
import bokeh.plotting as bp

def get_df(data, wi, wj=None):
    """
    Obtain the document frequency
    :param data: document vocabulary matrix
    :param wi: word index w_i
    :param wj: word index w_j
    :return: document frequency for word w_i , w_i ∩ w_j
    """
    if wj is None:
        return torch.where(data[:, wi] > 0, 1, 0).sum(-1)
    else:
        df_wi = torch.where(data[:, wi] > 0, 1, 0)
        df_wj = torch.where(data[:, wj] > 0, 1, 0)
        return df_wj.sum(-1), (df_wi & df_wj).sum(-1)


def get_topic_coherence(beta, data):
    D = torch.tensor(len(data)) ## number of docs...data is list of documents
    TC = []
    num_topics = len(beta)
    counter = 0
    for k in range(num_topics):
        top_10 = list(torch.flip(beta[k].argsort()[-11:],[0]))
        TC_k = 0
        counter = 0
        for i, word in enumerate(top_10):
            D_wi = get_df(data, word)
            j = i + 1
            tmp = 0
            while j < len(top_10) and j > i:
                D_wj, D_wi_wj = get_df(data, word, top_10[j])
                if D_wi_wj == 0:
                    f_wi_wj = -1
                else:
                    f_wi_wj = -1 + (torch.log(D_wi)+torch.log(D_wj)-2.0*torch.log(D))/(torch.log(D_wi_wj)-torch.log(D))
                tmp += f_wi_wj
                j += 1
                counter += 1
            TC_k += tmp 
        TC.append(TC_k.detach().cpu().numpy())
    TC = np.mean(TC) / counter
    #print('Topic coherence is: {}'.format(TC))
    return TC


def visualize(docs, _lda_keys, topics, theta):
    tsne_model = TSNE(n_components=2, verbose=1, random_state=0, angle=.99, init='pca')
    # project to 2D
    tsne_lda = tsne_model.fit_transform(theta)
    colormap = []
    for name, hex in matplotlib.colors.cnames.items():
        colormap.append(hex)

    colormap = colormap[:len(theta[0, :])]
    colormap = np.array(colormap)

    title = '20 newsgroups TE embedding V viz'
    num_example = len(docs)

    plot_lda = bp.figure(plot_width=1400, plot_height=1100,
                     title=title,
                     tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
                     x_axis_type=None, y_axis_type=None, min_border=1)

    plt.scatter(x=tsne_lda[:, 0], y=tsne_lda[:, 1],
                 color=colormap[_lda_keys][:num_example])
    plt.show()

def get_rnn_inp(data_batch,times_batch, num_times, vocab_size):
    rnn_input = torch.zeros(num_times, vocab_size).to(device)
    cnt = torch.zeros(num_times, ).to(device)
    for t in range(num_times):
        tmp = (times_batch == t).nonzero()
        docs = data_batch[tmp].squeeze().sum(0)
        rnn_input[t] += docs
        cnt[t] += tmp.shape[0]
    rnn_input = rnn_input / cnt.unsqueeze(1)
    return rnn_input
    
    
# def get_rnn_input(dataloader, num_times, vocab_size):
#     # TxV
#     rnn_input = torch.zeros(num_times, vocab_size).to(device)
#     # times count
#     cnt = torch.zeros(num_times, ).to(device)
#     # create data loader
#     for data in dataloader:
#         data_batch = cvz[data['index'].long() - 1,:].to(device)
#         times_batch = ts[data['index'].long() - 1].to(device)
#         for t in range(num_times):
#             # check times
#             tmp = (times_batch == t).nonzero()
#             # sum the vocabulary in time t
#             docs = data_batch[tmp].squeeze().sum(0)
#             # feed in the vocabulary count in time t
#             rnn_input[t] += docs
#             # sum up the count
#             cnt[t] += tmp.shape[0]
#             # check the epoch
#     rnn_input = rnn_input / cnt.unsqueeze(1)
#     return rnn_input

In [ ]:
import csv
import itertools
import numpy as np
import os
import re
import pandas as pd
import pickle
import random
import string
from tqdm import trange
from scipy import sparse
from scipy.io import savemat, loadmat
from sklearn.feature_extraction.text import CountVectorizer

# Maximum / minimum document frequency
max_df = 0.5
min_df = 100  # choose desired value for min_df

# read data
# # un debates
# data = pd.read_csv('/content/un-general-debates.csv')
# data = data[['session', 'year', 'country', 'text']]
# data.sort_values('year', ascending=True, inplace=True)
# docs = data.text.to_numpy()  ## bows
# neurips
# data = pd.read_csv('/content/papers.csv')
# data = data[~data.full_text.isnull()]
# docs = data.full_text.values  ## bows
# data.sort_values('year', ascending=True, inplace=True)
# US-Presidental Speeches
data = pd.read_csv('/content/presidential_speeches.csv')
data.sort_values('Date', ascending=True, inplace=True)
data = data[~data.Transcript.isnull()]
docs = data.Transcript.values
data['year']= pd.to_datetime(data.Date).dt.year

# unique timestamp
times = data.year.unique()
times.sort()

# timestamp input
ts = torch.from_numpy(data.year.to_numpy()).to(device)  ## timestamp
ts = (ts==ts.unique()[:,None]).nonzero().transpose(1,0)[0].to(device)
timestamps = data.year.values
# Read stopwords
with open('/content/stops.txt', 'r') as f:
    stops = f.read().split('\n')
    
print(f'Total document {len(docs)}')
#times_rank = (torch.tensor(ts)==torch.from_numpy(np.unique(ts))[:,None]).nonzero().T[0,:]

# Create count vectorizer
print('counting document frequency of words...')

def contains_punctuation(w):
    return any(char in string.punctuation for char in w)

def contains_numeric(w):
    return any(char.isdigit() for char in w)

# document preprocessing
init_docs = [re.findall(r'''[\w']+|[.,!?;-~{}`´_<=>:/@*()&'$%#"]|[\n]+''', doc) for doc in docs]
init_docs = [[w.lower() for w in init_docs[doc] if not contains_punctuation(w)] for doc in range(len(init_docs))]
init_docs = [[w for w in init_docs[doc] if not contains_numeric(w)] for doc in range(len(init_docs))]
init_docs = [[w for w in init_docs[doc] if len(w) > 1] for doc in range(len(init_docs))]
init_docs = [" ".join(init_docs[doc]) for doc in range(len(init_docs))]

# cvectorizer = CountVectorizer(min_df=min_df, max_df=max_df, stop_words=frozenset(stops))

# cvz = cvectorizer.fit_transform(init_docs)#.sign()
# cvz = torch.from_numpy(cvz.todense())

#print(f'documents={cvz.shape[0]};vocabulary={cvz.shape[1]};tokens={cvz.sum()}')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Total document 991
counting document frequency of words...


In [ ]:

# Read raw data
print('reading raw data...')
# docs = []
# not_found = []
# timestamps = []
# for (pid, tt) in zip(all_pids, all_timestamps):
#     path_read = 'raw/acl_abstracts/acl_data-combined/all_papers'
#     path_read = os.path.join(path_read, pid + '.txt')
#     if not os.path.isfile(path_read):
#         not_found.append(pid)
#     else:
#         with open(path_read, 'rb') as f:
#             doc = f.read().decode('utf-8', 'ignore')
#             doc = doc.lower().replace('\n', ' ').replace("’", " ").replace("'", " ").translate(str.maketrans(string.punctuation + "0123456789", ' '*len(string.punctuation + "0123456789"))).split()
#         doc = [remove_not_printable(w) for w in doc if len(w)>1]
#         if len(doc) > 1:
#             doc = " ".join(doc)
#             docs.append(doc)
#             timestamps.append(tt)

# Write as raw text
# print('writing to text file...')
# out_filename = './docs_processed.txt'
# print('writing to text file...')
# with open(out_filename, 'w') as f:
#     for line in docs:
#         f.write(line + '\n')

# Read stopwords
# with open('../input/stopwords/stops.txt', 'r') as f:
#     stops = f.read().split('\n')

# Create count vectorizer
print('counting document frequency of words...')
cvectorizer = CountVectorizer(min_df=min_df, max_df=max_df, stop_words=frozenset(stops))
cvz = cvectorizer.fit_transform(init_docs).sign()

# Get vocabulary
print('building the vocabulary...')
sum_counts = cvz.sum(axis=0)
v_size = sum_counts.shape[1]
sum_counts_np = np.zeros(v_size, dtype=int)
for v in range(v_size):
    sum_counts_np[v] = sum_counts[0,v]
word2id = dict([(w, cvectorizer.vocabulary_.get(w)) for w in cvectorizer.vocabulary_])
id2word = dict([(cvectorizer.vocabulary_.get(w), w) for w in cvectorizer.vocabulary_])
#del cvectorizer
print('  initial vocabulary size: {}'.format(v_size))

# Sort elements in vocabulary
idx_sort = np.argsort(sum_counts_np)
vocab_aux = [id2word[idx_sort[cc]] for cc in range(v_size)]

# Filter out stopwords (if any)
vocab_aux = [w for w in vocab_aux if w not in stops]
print('  vocabulary size after removing stopwords from list: {}'.format(len(vocab_aux)))

# Create dictionary and inverse dictionary
vocab = vocab_aux
del vocab_aux
word2id = dict([(w, j) for j, w in enumerate(vocab)])
id2word = dict([(j, w) for j, w in enumerate(vocab)])

# Create mapping of timestamps
all_times = sorted(set(timestamps))
time2id = dict([(t, i) for i, t in enumerate(all_times)])
id2time = dict([(i, t) for i, t in enumerate(all_times)])
time_list = [id2time[i] for i in range(len(all_times))]

# Split in train/test/valid
print('tokenizing documents and splitting into train/test/valid...')
num_docs = cvz.shape[0]
trSize = int(np.floor(0.8*num_docs))
tsSize = int(np.floor(0.15*num_docs))
vaSize = int(num_docs - trSize - tsSize)
#del cvz
idx_permute = np.random.permutation(num_docs).astype(int)

# Remove words not in train_data
# vocab = list(set([w for idx_d in range(trSize) for w in docs[idx_permute[idx_d]].split() if w in word2id]))
# word2id = dict([(w, j) for j, w in enumerate(vocab)])
# id2word = dict([(j, w) for j, w in enumerate(vocab)])
print('  vocabulary after removing words not in train: {}'.format(len(vocab)))

docs_tr = [[word2id[w] for w in docs[idx_permute[idx_d]].split() if w in word2id] for idx_d in range(trSize)]
timestamps_tr = [time2id[timestamps[idx_permute[idx_d]]] for idx_d in range(trSize)]
docs_ts = [[word2id[w] for w in docs[idx_permute[idx_d+trSize]].split() if w in word2id] for idx_d in range(tsSize)]
timestamps_ts = [time2id[timestamps[idx_permute[idx_d+trSize]]] for idx_d in range(tsSize)]
docs_va = [[word2id[w] for w in docs[idx_permute[idx_d+trSize+tsSize]].split() if w in word2id] for idx_d in range(vaSize)]
timestamps_va = [time2id[timestamps[idx_permute[idx_d+trSize+tsSize]]] for idx_d in range(vaSize)]

print('  number of documents (train): {} [this should be equal to {} and {}]'.format(len(docs_tr), trSize, len(timestamps_tr)))
print('  number of documents (test): {} [this should be equal to {} and {}]'.format(len(docs_ts), tsSize, len(timestamps_ts)))
print('  number of documents (valid): {} [this should be equal to {} and {}]'.format(len(docs_va), vaSize, len(timestamps_va)))

# Remove empty documents
print('removing empty documents...')

def remove_empty(in_docs, in_timestamps):
    out_docs = []
    out_timestamps = []
    for ii, doc in enumerate(in_docs):
        if(doc!=[]):
            out_docs.append(doc)
            out_timestamps.append(in_timestamps[ii])
    return out_docs, out_timestamps

def remove_by_threshold(in_docs, in_timestamps, thr):
    out_docs = []
    out_timestamps = []
    for ii, doc in enumerate(in_docs):
        if(len(doc)>thr):
            out_docs.append(doc)
            out_timestamps.append(in_timestamps[ii])
    return out_docs, out_timestamps

docs_tr, timestamps_tr = remove_empty(docs_tr, timestamps_tr)
docs_ts, timestamps_ts = remove_empty(docs_ts, timestamps_ts)
docs_va, timestamps_va = remove_empty(docs_va, timestamps_va)

# Remove test documents with length=1
docs_ts, timestamps_ts = remove_by_threshold(docs_ts, timestamps_ts, 1)

# Split test set in 2 halves
print('splitting test documents in 2 halves...')
docs_ts_h1 = [[w for i,w in enumerate(doc) if i<=len(doc)/2.0-1] for doc in docs_ts]
docs_ts_h2 = [[w for i,w in enumerate(doc) if i>len(doc)/2.0-1] for doc in docs_ts]

# Getting lists of words and doc_indices
print('creating lists of words...')

def create_list_words(in_docs):
    return [x for y in in_docs for x in y]

words_tr = create_list_words(docs_tr)
words_ts = create_list_words(docs_ts)
words_ts_h1 = create_list_words(docs_ts_h1)
words_ts_h2 = create_list_words(docs_ts_h2)
words_va = create_list_words(docs_va)

print('  len(words_tr): ', len(words_tr))
print('  len(words_ts): ', len(words_ts))
print('  len(words_ts_h1): ', len(words_ts_h1))
print('  len(words_ts_h2): ', len(words_ts_h2))
print('  len(words_va): ', len(words_va))

# Get doc indices
print('getting doc indices...')

def create_doc_indices(in_docs):
    aux = [[j for i in range(len(doc))] for j, doc in enumerate(in_docs)]
    return [int(x) for y in aux for x in y]

doc_indices_tr = create_doc_indices(docs_tr)
doc_indices_ts = create_doc_indices(docs_ts)
doc_indices_ts_h1 = create_doc_indices(docs_ts_h1)
doc_indices_ts_h2 = create_doc_indices(docs_ts_h2)
doc_indices_va = create_doc_indices(docs_va)

print('  len(np.unique(doc_indices_tr)): {} [this should be {}]'.format(len(np.unique(doc_indices_tr)), len(docs_tr)))
print('  len(np.unique(doc_indices_ts)): {} [this should be {}]'.format(len(np.unique(doc_indices_ts)), len(docs_ts)))
print('  len(np.unique(doc_indices_ts_h1)): {} [this should be {}]'.format(len(np.unique(doc_indices_ts_h1)), len(docs_ts_h1)))
print('  len(np.unique(doc_indices_ts_h2)): {} [this should be {}]'.format(len(np.unique(doc_indices_ts_h2)), len(docs_ts_h2)))
print('  len(np.unique(doc_indices_va)): {} [this should be {}]'.format(len(np.unique(doc_indices_va)), len(docs_va)))

# Number of documents in each set
n_docs_tr = len(docs_tr)
n_docs_ts = len(docs_ts)
n_docs_ts_h1 = len(docs_ts_h1)
n_docs_ts_h2 = len(docs_ts_h2)
n_docs_va = len(docs_va)

# Remove unused variables
del docs_tr
del docs_ts
del docs_ts_h1
del docs_ts_h2
del docs_va

# Create bow representation
print('creating bow representation...')

def create_bow(doc_indices, words, n_docs, vocab_size):
    return sparse.coo_matrix(([1]*len(doc_indices),(doc_indices, words)), shape=(n_docs, vocab_size)).tocsr()

bow_tr = create_bow(doc_indices_tr, words_tr, n_docs_tr, len(vocab))
bow_ts = create_bow(doc_indices_ts, words_ts, n_docs_ts, len(vocab))
bow_ts_h1 = create_bow(doc_indices_ts_h1, words_ts_h1, n_docs_ts_h1, len(vocab))
bow_ts_h2 = create_bow(doc_indices_ts_h2, words_ts_h2, n_docs_ts_h2, len(vocab))
bow_va = create_bow(doc_indices_va, words_va, n_docs_va, len(vocab))

del words_tr
del words_ts
del words_ts_h1
del words_ts_h2
del words_va
del doc_indices_tr
del doc_indices_ts
del doc_indices_ts_h1
del doc_indices_ts_h2
del doc_indices_va

# Write files for LDA C++ code
def write_lda_file(filename, timestamps_in, time_list_in, bow_in):
    idxSort = np.argsort(timestamps_in)
    
    with open(filename, "w") as f:
        for row in idxSort:
            x = bow_in.getrow(row)
            n_elems = x.count_nonzero()
            f.write(str(n_elems))
            if(n_elems != len(x.indices) or n_elems != len(x.data)):
                raise ValueError("[ERR] THIS SHOULD NOT HAPPEN")
            for ii, dd in zip(x.indices, x.data):
                f.write(' ' + str(ii) + ':' + str(dd))
            f.write('\n')
            
    with open(filename.replace("-mult", "-seq"), "w") as f:
        f.write(str(len(time_list_in)) + '\n')
        for idx_t, _ in enumerate(time_list_in):
            n_elem = len([t for t in timestamps_in if t==idx_t])
            f.write(str(n_elem) + '\n')
            

path_save = './min_df_' + str(min_df) + '/'
if not os.path.isdir(path_save):
    os.system('mkdir -p ' + path_save)

# Write files for LDA C++ code
print('saving LDA files for C++ code...')
write_lda_file(path_save + 'dtm_tr-mult.dat', timestamps_tr, time_list, bow_tr)
write_lda_file(path_save + 'dtm_ts-mult.dat', timestamps_ts, time_list, bow_ts)
write_lda_file(path_save + 'dtm_ts_h1-mult.dat', timestamps_ts, time_list, bow_ts_h1)
write_lda_file(path_save + 'dtm_ts_h2-mult.dat', timestamps_ts, time_list, bow_ts_h2)
write_lda_file(path_save + 'dtm_va-mult.dat', timestamps_va, time_list, bow_va)

# Also write the vocabulary and timestamps
with open(path_save + 'vocab.txt', "w") as f:
    for v in vocab:
        f.write(v + '\n')

with open(path_save + 'timestamps.txt', "w") as f:
    for t in time_list:
        f.write(str(t) + '\n')

with open(path_save + 'vocab.pkl', 'wb') as f:
    pickle.dump(vocab, f)
del vocab

with open(path_save + 'timestamps.pkl', 'wb') as f:
    pickle.dump(time_list, f)

# Save timestamps alone
savemat(path_save + 'bow_tr_timestamps', {'timestamps': timestamps_tr}, do_compression=True)
savemat(path_save + 'bow_ts_timestamps', {'timestamps': timestamps_ts}, do_compression=True)
savemat(path_save + 'bow_va_timestamps', {'timestamps': timestamps_va}, do_compression=True)

# Split bow intro token/value pairs
print('splitting bow intro token/value pairs and saving to disk...')

def split_bow(bow_in, n_docs):
    indices = [[w for w in bow_in[doc,:].indices] for doc in range(n_docs)]
    counts = [[c for c in bow_in[doc,:].data] for doc in range(n_docs)]
    return indices, counts

bow_tr_tokens, bow_tr_counts = split_bow(bow_tr, n_docs_tr)
savemat(path_save + 'bow_tr_tokens', {'tokens': bow_tr_tokens}, do_compression=True)
savemat(path_save + 'bow_tr_counts', {'counts': bow_tr_counts}, do_compression=True)
del bow_tr
del bow_tr_tokens
del bow_tr_counts

bow_ts_tokens, bow_ts_counts = split_bow(bow_ts, n_docs_ts)
savemat(path_save + 'bow_ts_tokens', {'tokens': bow_ts_tokens}, do_compression=True)
savemat(path_save + 'bow_ts_counts', {'counts': bow_ts_counts}, do_compression=True)
del bow_ts
del bow_ts_tokens
del bow_ts_counts

bow_ts_h1_tokens, bow_ts_h1_counts = split_bow(bow_ts_h1, n_docs_ts_h1)
savemat(path_save + 'bow_ts_h1_tokens', {'tokens': bow_ts_h1_tokens}, do_compression=True)
savemat(path_save + 'bow_ts_h1_counts', {'counts': bow_ts_h1_counts}, do_compression=True)
del bow_ts_h1
del bow_ts_h1_tokens
del bow_ts_h1_counts

bow_ts_h2_tokens, bow_ts_h2_counts = split_bow(bow_ts_h2, n_docs_ts_h2)
savemat(path_save + 'bow_ts_h2_tokens', {'tokens': bow_ts_h2_tokens}, do_compression=True)
savemat(path_save + 'bow_ts_h2_counts', {'counts': bow_ts_h2_counts}, do_compression=True)
del bow_ts_h2
del bow_ts_h2_tokens
del bow_ts_h2_counts

bow_va_tokens, bow_va_counts = split_bow(bow_va, n_docs_va)
savemat(path_save + 'bow_va_tokens', {'tokens': bow_va_tokens}, do_compression=True)
savemat(path_save + 'bow_va_counts', {'counts': bow_va_counts}, do_compression=True)
del bow_va
del bow_va_tokens
del bow_va_counts

print('Data ready !!')
print('*************')

reading raw data...
counting document frequency of words...
building the vocabulary...
  initial vocabulary size: 1891
  vocabulary size after removing stopwords from list: 1891
tokenizing documents and splitting into train/test/valid...
  vocabulary after removing words not in train: 1891
  number of documents (train): 792 [this should be equal to 792 and 792]
  number of documents (test): 148 [this should be equal to 148 and 148]
  number of documents (valid): 51 [this should be equal to 51 and 51]
removing empty documents...
splitting test documents in 2 halves...
creating lists of words...
  len(words_tr):  495683
  len(words_ts):  83149
  len(words_ts_h1):  41536
  len(words_ts_h2):  41613
  len(words_va):  26589
getting doc indices...
  len(np.unique(doc_indices_tr)): 792 [this should be 792]
  len(np.unique(doc_indices_ts)): 148 [this should be 148]
  len(np.unique(doc_indices_ts_h1)): 148 [this should be 148]
  len(np.unique(doc_indices_ts_h2)): 148 [this should be 148]
  len(n

/usr/local/lib/python3.7/dist-packages/scipy/io/matlab/mio5.py:450: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  narr = np.asanyarray(source)


Data ready !!
*************


In [ ]:
from sklearn.manifold import TSNE
import torch 
import numpy as np
import bokeh.plotting as bp

from bokeh.plotting import save
from bokeh.models import HoverTool
import matplotlib.pyplot as plt 
import matplotlib 

tiny = 1e-6

def _reparameterize(mu, logvar, num_samples):
    """Applies the reparameterization trick to return samples from a given q"""
    std = torch.exp(0.5 * logvar) 
    bsz, zdim = logvar.size()
    eps = torch.randn(num_samples, bsz, zdim).to(mu.device)
    mu = mu.unsqueeze(0)
    std = std.unsqueeze(0)
    res = eps.mul_(std).add_(mu)
    return res

def log_gaussian(z, mu=None, logvar=None):
    sz = z.size()
    d = z.size(2)
    bsz = z.size(1)
    if mu is None or logvar is None:
        mu = torch.zeros(bsz, d).to(z.device)
        logvar = torch.zeros(bsz, d).to(z.device)
    mu = mu.unsqueeze(0)
    logvar = logvar.unsqueeze(0)
    var = logvar.exp()
    log_density = ((z - mu)**2 / (var+tiny)).sum(2) # b
    log_det = logvar.sum(2) # b
    log_density = log_density + log_det + d*np.log(2*np.pi)
    return -0.5*log_density

def logsumexp(x, dim=0):
    d = torch.max(x, dim)[0]   
    if x.dim() == 1:
        return torch.log(torch.exp(x - d).sum(dim)) + d
    else:
        return torch.log(torch.exp(x - d.unsqueeze(dim).expand_as(x)).sum(dim) + tiny) + d

def flatten_docs(docs): #to get words and doc_indices
    words = [x for y in docs for x in y]
    doc_indices = [[j for _ in doc] for j, doc in enumerate(docs)]
    doc_indices = [x for y in doc_indices for x in y]
    return words, doc_indices
    
def onehot(data, min_length):
    return list(np.bincount(data, minlength=min_length))

def nearest_neighbors(word, embeddings, vocab, num_words):
    vectors = embeddings.cpu().numpy() 
    index = vocab.index(word)
    query = embeddings[index].cpu().numpy() 
    ranks = vectors.dot(query).squeeze()
    denom = query.T.dot(query).squeeze()
    denom = denom * np.sum(vectors**2, 1)
    denom = np.sqrt(denom)
    ranks = ranks / denom
    mostSimilar = []
    [mostSimilar.append(idx) for idx in ranks.argsort()[::-1]]
    nearest_neighbors = mostSimilar[:num_words]
    nearest_neighbors = [vocab[comp] for comp in nearest_neighbors]
    return nearest_neighbors

def visualize(docs, _lda_keys, topics, theta):
    tsne_model = TSNE(n_components=2, verbose=1, random_state=0, angle=.99, init='pca')
    # project to 2D
    tsne_lda = tsne_model.fit_transform(theta)
    colormap = []
    for name, hex in matplotlib.colors.cnames.items():
        colormap.append(hex)

    colormap = colormap[:len(theta[0, :])]
    colormap = np.array(colormap)

    title = '20 newsgroups TE embedding V viz'
    num_example = len(docs)

    plot_lda = bp.figure(plot_width=1400, plot_height=1100,
                     title=title,
                     tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
                     x_axis_type=None, y_axis_type=None, min_border=1)

    plt.scatter(x=tsne_lda[:, 0], y=tsne_lda[:, 1],
                 color=colormap[_lda_keys][:num_example])
    plt.show()

import os
import pickle

import numpy as np
import scipy.io
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def _fetch(path, name):
    if name == 'train':
        token_file = os.path.join(path, 'bow_tr_tokens')
        count_file = os.path.join(path, 'bow_tr_counts')
    elif name == 'valid':
        token_file = os.path.join(path, 'bow_va_tokens')
        count_file = os.path.join(path, 'bow_va_counts')
    else:
        token_file = os.path.join(path, 'bow_ts_tokens')
        count_file = os.path.join(path, 'bow_ts_counts')
    tokens = scipy.io.loadmat(token_file)['tokens'].squeeze()
    counts = scipy.io.loadmat(count_file)['counts'].squeeze()
    if name == 'test':
        token_1_file = os.path.join(path, 'bow_ts_h1_tokens')
        count_1_file = os.path.join(path, 'bow_ts_h1_counts')
        token_2_file = os.path.join(path, 'bow_ts_h2_tokens')
        count_2_file = os.path.join(path, 'bow_ts_h2_counts')
        tokens_1 = scipy.io.loadmat(token_1_file)['tokens'].squeeze()
        counts_1 = scipy.io.loadmat(count_1_file)['counts'].squeeze()
        tokens_2 = scipy.io.loadmat(token_2_file)['tokens'].squeeze()
        counts_2 = scipy.io.loadmat(count_2_file)['counts'].squeeze()
        return {'tokens': tokens, 'counts': counts, 'tokens_1': tokens_1, 'counts_1': counts_1, 'tokens_2': tokens_2,
                'counts_2': counts_2}
    return {'tokens': tokens, 'counts': counts}


def _fetch_temporal(path, name):
    if name == 'train':
        token_file = os.path.join(path, 'bow_tr_tokens')
        count_file = os.path.join(path, 'bow_tr_counts')
        time_file = os.path.join(path, 'bow_tr_timestamps')
    elif name == 'valid':
        token_file = os.path.join(path, 'bow_va_tokens')
        count_file = os.path.join(path, 'bow_va_counts')
        time_file = os.path.join(path, 'bow_va_timestamps')
    else:
        token_file = os.path.join(path, 'bow_ts_tokens')
        count_file = os.path.join(path, 'bow_ts_counts')
        time_file = os.path.join(path, 'bow_ts_timestamps')
    tokens = scipy.io.loadmat(token_file)['tokens'].squeeze()
    counts = scipy.io.loadmat(count_file)['counts'].squeeze()
    times = scipy.io.loadmat(time_file)['timestamps'].squeeze()
    if name == 'test':
        token_1_file = os.path.join(path, 'bow_ts_h1_tokens')
        count_1_file = os.path.join(path, 'bow_ts_h1_counts')
        token_2_file = os.path.join(path, 'bow_ts_h2_tokens')
        count_2_file = os.path.join(path, 'bow_ts_h2_counts')
        tokens_1 = scipy.io.loadmat(token_1_file)['tokens'].squeeze()
        counts_1 = scipy.io.loadmat(count_1_file)['counts'].squeeze()
        tokens_2 = scipy.io.loadmat(token_2_file)['tokens'].squeeze()
        counts_2 = scipy.io.loadmat(count_2_file)['counts'].squeeze()
        return {'tokens': tokens, 'counts': counts, 'times': times,
                'tokens_1': tokens_1, 'counts_1': counts_1,
                'tokens_2': tokens_2, 'counts_2': counts_2}
    return {'tokens': tokens, 'counts': counts, 'times': times}


def get_data(path, temporal=False):
    ### load vocabulary
    with open(os.path.join(path, 'vocab.pkl'), 'rb') as f:
        vocab = pickle.load(f)

    if not temporal:
        train = _fetch(path, 'train')
        valid = _fetch(path, 'valid')
        test = _fetch(path, 'test')
    else:
        train = _fetch_temporal(path, 'train')
        valid = _fetch_temporal(path, 'valid')
        test = _fetch_temporal(path, 'test')

    return vocab, train, valid, test


def get_batch(tokens, counts, ind, vocab_size, temporal=False, times=None):
    """fetch input data by batch."""
    batch_size = len(ind)
    data_batch = np.zeros((batch_size, vocab_size))
    if temporal:
        times_batch = np.zeros((batch_size,))
    for i, doc_id in enumerate(ind):
        doc = tokens[doc_id]
        count = counts[doc_id]
        if temporal:
            timestamp = times[doc_id]
            times_batch[i] = timestamp
        L = count.shape[1]
        if len(doc) == 1:
            doc = [doc.squeeze()]
            count = [count.squeeze()]
        else:
            doc = doc.squeeze()
            count = count.squeeze()
        if doc_id != -1:
            for j, word in enumerate(doc):
                data_batch[i, word] = count[j]
    data_batch = torch.from_numpy(data_batch).float().to(device)
    if temporal:
        times_batch = torch.from_numpy(times_batch).to(device)
        return data_batch, times_batch
    return data_batch

def get_rnn_input(tokens, counts, times, num_times, vocab_size, num_docs):
    #(data_batch,times_batch, num_times, vocab_size):
    ind = torch.randperm(num_docs).to(device)
    data_batch, times_batch = get_batch(tokens, counts, ind, vocab_size, temporal=True, times=times)
    rnn_input = torch.zeros(num_times, vocab_size).to(device)
    cnt = torch.zeros(num_times, ).to(device)
    for t in range(num_times):
        tmp = (times_batch == t).nonzero()
        docs = data_batch[tmp].squeeze().sum(0)
        rnn_input[t] += docs
        cnt[t] += tmp.shape[0]
    rnn_input = rnn_input / cnt.unsqueeze(1)
    return rnn_input

# def get_rnn_input(tokens, counts, times, num_times, vocab_size, num_docs):
#     indices = torch.randperm(num_docs)
#     indices = torch.split(indices, 1000)
#     rnn_input = torch.zeros(num_times, vocab_size).to(device)
#     cnt = torch.zeros(num_times, ).to(device)
#     for idx, ind in enumerate(indices):
#         data_batch, times_batch = get_batch(tokens, counts, ind, vocab_size, temporal=True, times=times)
#         for t in range(num_times):
#             tmp = (times_batch == t).nonzero()
#             docs = data_batch[tmp].squeeze().sum(0)
#             rnn_input[t] += docs
#             cnt[t] += len(tmp)
#         if idx % 20 == 0:
#             print('idx: {}/{}'.format(idx, len(indices)))
#     rnn_input = rnn_input / cnt.unsqueeze(1)
#     return rnn_input


In [ ]:
print('Getting vocabulary ...')
data_file = os.path.join('./', 'min_df_{}'.format(100))
vocab, train, valid, test = get_data(data_file, temporal=True)
vocab_size = len(vocab)

# 1. training data
print('Getting training data ...')
train_tokens = train['tokens']
train_counts = train['counts']
train_times = train['times']
num_times = train_times.max()-train_times.min()+1#len(np.unique(train_times))
num_docs_train = len(train_tokens)

batch_size=1024
test_ratio = 0.8
train_size = int(np.floor(test_ratio * len(train_tokens)))
test_size = int(np.floor(0.15*len(train_tokens)))
#test_size = test_size-(test_size%batch_size)
valid_size = len(train_tokens)-train_size-test_size


# 2. dev set
print('Getting validation data ...')
valid_tokens = valid['tokens']
valid_counts = valid['counts']
valid_times = valid['times']
num_docs_valid = len(valid_tokens)
valid_rnn_inp = get_rnn_input(
    valid_tokens, valid_counts, valid_times, num_times, vocab_size, valid_size).nan_to_num()

# 3. test data
print('Getting testing data ...')
test_tokens = test['tokens']
test_counts = test['counts']
test_times = test['times']
num_docs_test = len(test_tokens)
test_rnn_inp = get_rnn_input(
    test_tokens, test_counts, test_times, num_times, vocab_size, test_size).nan_to_num()

train_rnn_inp = get_rnn_input(
    train_tokens, train_counts, train_times, num_times, vocab_size, train_size).nan_to_num()

Getting vocabulary ...
Getting training data ...
Getting validation data ...
Getting testing data ...


In [ ]:
train_cvz, train_ts = get_batch(
            train_tokens, train_counts, torch.tensor(range(train_size)).to(device), vocab_size, temporal=True, times=train_times)
valid_cvz, valid_ts = get_batch(
            valid_tokens, valid_counts, torch.tensor(range(valid_size)).to(device), vocab_size, temporal=True, times=valid_times)
test_cvz, test_ts = get_batch(
            test_tokens, test_counts, torch.tensor(range(test_size)).to(device), vocab_size, temporal=True, times=test_times)

In [ ]:
# batch_size=1024
# test_ratio = 0.8
# train_size = int(np.floor(test_ratio * len(cvz)))
# test_size = int(np.floor(0.15*len(cvz)))
# #test_size = test_size-(test_size%batch_size)
# valid_size = len(cvz)-train_size-test_size

# print(train_size, test_size, valid_size)

# cvz = cvz.float().to(device)
# ts = ts.to(device)

# shuf_ind = torch.randperm(cvz.shape[0])
# cvz = cvz[shuf_ind]
# ts = ts[shuf_ind]

# train_cvz, test_cvz, valid_cvz = torch.split(cvz, [train_size, test_size, valid_size])
# train_ts,  test_ts,  valid_ts  = torch.split(ts,  [train_size, test_size, valid_size])
# # shuffle
# indexes = torch.randperm(train_cvz.shape[0])
# train_cvz = train_cvz[indexes]
# train_ts = train_ts[indexes]

In [ ]:
from torch.distributions.constraints import positive
import torch
import math

def evaluate(model):
    model.eval()
    alpha = model.mu_q_alpha.clone().contiguous()    # KxTxL
    alpha = alpha.permute(1, 0, 2)
    beta = model.get_beta(alpha)
    beta = beta[:,:,:-3]
    cnt = 0
    tc = 0
    for time in range(0, beta.shape[0]):
        beta_t = beta[time,:,:]
        cnt+=1
        tc+=get_topic_coherence(beta_t, cvz)
    tc/=cnt
    print(f'tc: {tc}')



torch.set_default_tensor_type('torch.FloatTensor')
# get index first
# train_rnn_inp = get_rnn_inp(train_cvz.to(device), train_ts.to(device), len(times), train_cvz.shape[1])
# test_rnn_inp = get_rnn_inp(test_cvz.to(device), test_ts.to(device), len(times), test_cvz.shape[1])
# valid_rnn_inp = get_rnn_inp(valid_cvz.to(device), valid_ts.to(device), len(times), valid_cvz.shape[1])
# define model
model = TETM(vocab_size=cvz.shape[1],
             num_topics=30,
             num_times=len(times),
             hidden=800,
             dropout=0.0,
             delta=0.005,
             emb_type='NN',
             useEmbedding=True,
             trainEmbedding=True,
             rho_size=300,
             eta_size=128,
             data_size=train_rnn_inp.shape[0]
            )

#print_top_words(beta[:,:,:-3],vocab)
def _diversity_helper(beta, num_tops):
    list_w = torch.zeros((int(beta.shape[0]), num_tops))
    for k in range(int(beta.shape[0])):
        gamma = beta[k, :]
        top_words = gamma.argsort()[-num_tops:]
        list_w[k, :] = top_words
    list_w = list_w.reshape(-1)
    n_unique = len(list_w.unique())
    diversity = n_unique / (beta.shape[0] * num_tops)
    return diversity


# https://stackoverflow.com/questions/49201236/check-the-total-number-of-parameters-in-a-pytorch-model
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params
# print model parameters
# count_parameters(model)
print(model)

num_epochs=1000
bar = trange(num_epochs)

# dataloader loop
# TODO save the loss/batch-loss
# - reconstruction loss
recon_loss_trace = []
# - kl-loss
kl_loss_trace = []
# - transformer loss
trans_loss_trace = []
# - combined loss, can be done with post-processing
combined_loss_trace = []
# - validation perplexity
val_ppl_trace = []

num_batches = int(math.ceil(train_cvz.shape[0] / batch_size))
optim = torch.optim.Adam(model.parameters(), lr=5e-3, weight_decay=1e-6)
for epoch in bar:
    batch_recon_loss = []
    batch_kl_loss = []
    batch_trans_loss = []
    for i in range(num_batches):
        model.train()
        model.zero_grad()
        optim.zero_grad()

        if (i + 1) * batch_size > len(train_cvz):
            batch_docs = train_cvz[i * batch_size:, :]
            time_batch = train_ts[i * batch_size:]
        else:    
            batch_docs = train_cvz[i * batch_size:(i + 1) * batch_size, :]
            time_batch = train_ts[i * batch_size:(i + 1) * batch_size]
        batch_docs = batch_docs.nan_to_num()
        # normalize batch
        sums = batch_docs.sum(1).unsqueeze(1)
        normalized_data_batch = batch_docs #/ sums
        # Calculate loss of transformer model
        recon_loss, kl_alpha, kl_eta, kl_theta,kld_eta_gp = model(batch_docs, normalized_data_batch, time_batch, 
                                                       train_rnn_inp,len(train_cvz))
        # scale the product
        bsz = batch_docs.size(0)
        #coeff = len(cvz) / bsz
        kl_loss = recon_loss + kl_alpha + kl_theta + kl_eta - 0.5*kld_eta_gp
        kl_loss = kl_loss.sum()
        # optimizer
        batch_loss = kl_loss # + recon_loss
        bar.set_postfix(recon='{:.5e}'.format(recon_loss),
                        alpha='{:.2e}'.format(kl_alpha),
                        theta='{:.2e}'.format(kl_theta.sum()),
                        eta='{:.2e}'.format(kl_eta.sum()))

        # gradient step
        batch_loss.backward()
        optim.step()
        batch_loss.item()
#         batch_recon_loss.append(recon_loss.item())
        batch_kl_loss.append(kl_loss.item())
    # store the average loss
#     recon_loss_trace.append(batch_recon_loss)
    kl_loss_trace.append(batch_kl_loss)

 #   if epoch % 100 == 0 and epoch > 0:
        # topic coherence
#        evaluate(model)
    if epoch> 0 and epoch % 980 == 0:
        # KxTxL
        alpha = model.get_alpha()[0]
        beta = model.get_beta(alpha)
        print(beta.shape)

        cnt = 0
        tc = 0
        for time in range(0,beta.shape[0]):
            beta_t = beta[time,:,:]
            cnt+=1
            tc_k=get_topic_coherence(beta_t, train_cvz)
            print(tc_k)
            tc+=tc_k

        tc/=cnt
        print(f'tc: {tc}')

    model.eval()
    sums = valid_cvz.sum(1).unsqueeze(1)
    valid_cvz = valid_cvz.nan_to_num()
    normalized_valid_batch = valid_cvz.nan_to_num() #/ sums
    ppl = model.predict(valid_cvz, normalized_valid_batch, valid_ts.nan_to_num(), valid_rnn_inp.nan_to_num())
    print(f'Validation perplexity: {ppl}')
    val_ppl_trace.append(ppl)
    # KxTxL
    alpha = model.get_alpha()[0]
    beta = model.get_beta(alpha)
    td = 0
    for t in range(beta.shape[0]):
        d=_diversity_helper(beta[t],25)
        td+=d
    print(f'TD: {td/beta.shape[0]}')

# test_ppl = 0
# test_cnt = 0
# for data in test_loader:
#     d_batch, t_batch = cvz[data['index'] - 1, :], ts[data['index'] - 1]
#     test_ppl += model.predict(d_batch.nan_to_num(), t_batch.nan_to_num(), train_rnn_inp.nan_to_num())
#     test_cnt += 1
# # perplexity
# test_ppl /= test_cnt
# print(f'Test perplpexity: {test_ppl}')

In [ ]:
model.eval()
print(test_cvz.shape)
test_cvz = test_cvz.nan_to_num().float()  
normalized_test_batch = test_cvz #/ sums
ppl = model.predict(test_cvz, normalized_test_batch, test_ts, test_rnn_inp.nan_to_num())
print(f'Validation perplexity: {ppl}')

In [ ]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, num_workers=1, shuffle=False)
#test_rnn_inp = get_rnn_input(test_loader, len(ts.unique()),cvz.shape[1]).cuda().nan_to_num(0)
print(d_batch, t_batch, len(dataset))
    
test_ppl = 0
test_cnt = 0
for data in test_loader:
    d_batch, t_batch = cvz[data['index'] - 1, :].to(device), ts[data['index'] - 1].to(device)
    test_rnn_inp = get_rnn_inp(d_batch, t_batch, len(ts.unique()),cvz.shape[1]).cuda().nan_to_num(0)
    test_ppl += model.predict(d_batch, t_batch, test_rnn_inp)
    test_cnt += 1
# perplexity
test_ppl /= test_cnt
print(f'Test perplpexity: {test_ppl}')

Topic Coherence

In [ ]:
# output the result
def print_top_words(beta, feature_names, n_top_words=10):
    for k in range(beta.shape[1]):
        for t in range(0,beta.shape[0],5):
            print(
                ("Topic #%d@time=%d: " % (k,times[t]))
                + " ".join([feature_names[j]  for j in beta[t,k].argsort()[-n_top_words:]])
            )

print_top_words(beta[:,best_topics_rank,:-3],vocab)

In [ ]:
torch.save(model.state_dict(), 'model.pkt')
#Later to restore:
# model.load_state_dict(torch.load(filepath))
# model.eval()

In [ ]:
len(vocab)

f = open("test.txt","w")
f.write(' '.join(vocab))
f.close()

beta = model.get_beta_result()
torch.save(beta, 'beta.pt')
eta = model.get_eta_result()
torch.save(eta, 'eta.pt')

## Visualization

In [ ]:
beta.shape

In [ ]:
data_file = os.path.join('./', 'min_df_{}'.format(100))
vocab, train, valid, test = get_data(data_file, temporal=True)
# KxTxL
alpha = model.mu_q_alpha.clone().contiguous()
alpha = alpha.permute(1,0,2)
#alpha = model.get_alpha()[0]

beta = model.get_beta(alpha)#, torch.arange(0,len(times)))
# for bth in test_loader:
#     t_index = torch.cat([t_index, bth['index']], dim=0)
    
t_bows = test_cvz#cvz[t_index - 1,:]
t_times = test_ts#ts[t_index - 1]
eta = model.get_eta_result()
#eta, _ = model.get_mu(train_rnn_inp)
#eta = eta.detach().cpu().numpy()
theta, _ = model.get_theta(eta, t_bows, t_times)
# select best 5 topics
best_topics_rank = theta.sum(0).argsort()[-6:]
# custom topics
#best_topics_rank = theta.sum(0).argsort()[[0,1,9,13,16,19]].cuda().long()
# select best 4 words from those topics
best_vocab_rank = beta[:,best_topics_rank,:-3].sum(0).argsort(-1)[:,-5:]
# check beta
best_vocab = np.array(vocab)[best_vocab_rank.detach().cpu().numpy()]
# 
#eta = eta.detach().cpu().numpy()
eta = eta.softmax(-1).detach().cpu().numpy()
theta = theta.detach().cpu().numpy()

best_topics_rank, best_vocab_rank, best_vocab

In [ ]:
len(vocab)

In [ ]:
# selected_rank = [[2,3,5],[0,1,2],[0,4,5],[1,4,5],[0,1,2],[1,3,4]]
# selected_vocab_rank = best_vocab_rank.cpu().detach().numpy()[:,selected_rank]
# selected_vocab_format = torch.tensor(np.array([selected_vocab_rank[i,i,:] for i in range(6)]))
# selected_vocab = np.array(vocab)[selected_vocab_format]

In [ ]:
# best_topics_rank = torch.tensor([0,1,9,13,16,17])

In [ ]:
x = np.linspace(times[0],times[-1],times.shape[0])
result = beta[:,best_topics_rank, :].squeeze(1)[:,:,best_vocab_rank].detach().cpu().numpy()

plt.figure(figsize=(12, 8), dpi=80)

for i in range(1,7):
    plt.subplot(2,3,i)
    plt.plot(x,result[:,i-1,i-1,:],label=best_vocab[i-1])
    plt.legend(loc='upper left')
    plt.xlabel('Years')
    #plt.ylabel('Stack')

plt.savefig('scatter.png')
plt.show()

In [ ]:
test_ts.float()

In [ ]:
best_topics_rank.detach().cpu().numpy()

In [ ]:
model.decoder.fcrho.weight().shape, theta.shape
rho = model.decoder.fcrho.weight().detach().cpu().numpy()

In [ ]:
import matplotlib

import bokeh.plotting as bp

from bokeh.plotting import save
from bokeh.models import HoverTool

tsne_model = TSNE(n_components=2, verbose=1, random_state=0, angle=.99, init='pca')
# project to 2D
tsne_lda = tsne_model.fit_transform(theta)
colormap = []
for name, hex in matplotlib.colors.cnames.items():
    colormap.append(hex)

colormap = colormap[:len(theta[0, :])]
colormap = np.array(colormap)
title = '20 newsgroups TE embedding V viz'
plot_lda = bp.figure(plot_width=1400, plot_height=1100,
                 title=title,
                 tools="pan,wheel_zoom,box_zoom,reset,hover",
                 x_axis_type=None, y_axis_type=None, min_border=1)
plt.figure(figsize=(12,8))
plt.scatter(x=tsne_lda[:, 0], y=tsne_lda[:, 1],
                    color=colormap[theta.argsort(-1)[:,-1]%148])
plt.savefig('tsne1.png')
plt.show()

In [ ]:
s = lambda x: "#Topic: "+str(x)
list(map(s, best_topics_rank.detach().cpu().numpy().tolist()))

In [ ]:
s = list(map(" ".join, best_vocab))
# s = ['Topic 0: Graph',
#     'Topic 1: Cognitive Science',
#     'Topic 9: Embeddings',
#     'Topic 13: Visual & Signaling',
#     'Topic 16: Bayesian Statistics',
#     'Topic 17: Decision Theory',]

In [ ]:
import matplotlib.pyplot as plt
# time range
x = np.linspace(times[0],times[-1],times.shape[0])
# select the (topic, topic-words)
# topic word vocab
plt.figure(figsize=(12,8))
for i in range(0,6):
    plt.plot(x, eta.T[best_topics_rank.detach().cpu().numpy()[i],:], 
              labels=best_vocab)
             #label=s[i])
# plt.stackplot(x, eta.T[best_topics_rank.detach().cpu().numpy(),:], 
#               labels=best_vocab)
plt.legend(loc='upper left')
plt.xlabel('Year')
#plt.ylabel('Stack')
plt.title('Topic representation over time')
plt.savefig('topic_scatter.png')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
# time range
x = np.linspace(times[0],times[-1],times.shape[0])
# select the (topic, topic-words)
# topic word vocab
plt.figure(figsize=(12,8))
res = pd.DataFrame(eta.T[best_topics_rank.detach().cpu().numpy(),:].T).rolling(window=5).mean()#.plot(label="20SMA", legend=True)
custom_title = ['Theoretic bound','Latent variable model', 'Neural networks', 'Graph Theory', 'Bayesian statistics','Classification']
plt.stackplot(x, res.T,labels=custom_title)
#plt.stackplot(x, eta.T[best_topics_rank.detach().cpu().numpy(),:],labels=best_vocab)
plt.legend(loc='upper left')
plt.xlabel('Year')
#plt.ylabel('Stack')
plt.title('Topic representation over time')
plt.savefig('topic_stack_plot.png')
plt.show()

In [ ]:
pd.DataFrame(eta.T[best_topics_rank.detach().cpu().numpy(),:].T).rolling(window=5).mean().T

In [ ]:
!pip install pyvis

In [ ]:
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd

plt.figure(figsize=(16,12))
#sb.heatmap(pd.DataFrame(filtered_beta.T).corr(),vmin=-1,annot=True)
sb.heatmap(pd.DataFrame(model.L_Omega[30].cpu().numpy()),vmin=-1,annot=True)

In [ ]:
def get_top_words(k, top=5):
  # KxTxL
  alpha = model.mu_q_alpha.clone().contiguous()
  alpha = alpha.permute(1,0,2)
  #alpha = model.get_alpha()[0]

  beta = model.get_beta(alpha, torch.arange(0,len(times)))
  # select best 4 words from those topics
  best_vocab_rank = beta[:,k,:-3].sum(0).argsort(-1)[-top:]
  # check beta
  best_vocab = np.array(vocab)[best_vocab_rank.detach().cpu().numpy()]
  return best_vocab.tolist()

get_top_words(2)

In [ ]:
li = model.L_Omega[0][10].cpu().tolist()
li.pop(10)
len(li)

In [ ]:
from pyvis.network import Network

num_topics = 30
time_t = 26
# Let's visualize the result
g = Network(width=800, height=800, font_color="#333")
correl = model.L_Omega[time_t].reshape([-1]).cpu().tolist()
correl.sort()
top_tenth = num_topics * (num_topics - 1) // 10
top_tenth = correl[-num_topics - top_tenth]

for k in range(num_topics):
    label = "#{}".format(k)
    title= ' '.join(word for word in get_top_words(k, top=6))
    print('Topic', label, title)
    g.add_node(k, label=label, title=title, shape='ellipse')
    li = model.L_Omega[time_t][k].cpu().tolist()
    li.pop(k)
    for l, correlation in zip(range(k - 1), li):
        if correlation < top_tenth: continue
        g.add_edge(k, l, value=float(correlation), title='{:.02}'.format(correlation))

g.barnes_hut(gravity=-1000, spring_length=20)
g.show_buttons()
g.show("topic_network.html")

In [ ]:
eta = torch.from_numpy(eta).softmax(-1).detach().numpy()


x = np.linspace(times[0],times[-1],times.shape[0])

plt.figure(figsize=(12, 8), dpi=80)

for i in range(1,21):
    plt.subplot(5,4,i)
    plt.plot(x,eta[:,i-1])
    #plt.legend(loc='upper left')
    #plt.xlabel('Years')
    #plt.ylabel('Stack')

In [ ]:
data_file = os.path.join('./', 'min_df_{}'.format(100))
vocab, train, valid, test = get_data(data_file, temporal=True)
vocab, count

In [ ]:
# KxTxL
alpha = model.mu_q_alpha.clone().contiguous()
alpha = alpha.permute(1,0,2)
#alpha = model.get_alpha()[0]
beta = model.get_beta(alpha)
print(beta.shape)

cnt = 0
tc = 0
#beta = beta[:,:,:-3]
for time in range(0,beta.shape[0]):
    beta_t = beta[time,:,:]
    cnt+=1
    tc_k=get_topic_coherence(beta_t, valid_cvz)
    print(tc_k)
    tc+=tc_k

tc/=cnt
print(f'tc: {tc}')

#print_top_words(beta[:,:,:-3],vocab)
def _diversity_helper(beta, num_tops):
    list_w = torch.zeros((int(beta.shape[0]), num_tops))
    for k in range(int(beta.shape[0])):
        gamma = beta[k, :]
        top_words = gamma.argsort()[-num_tops:]
        list_w[k, :] = top_words
    list_w = list_w.reshape(-1)
    n_unique = len(list_w.unique())
    diversity = n_unique / (beta.shape[0] * num_tops)
    return diversity

td = 0
for t in range(beta.shape[0]):
    d=_diversity_helper(beta[t],25)
    td+=d
    print(d)
print(f'TD: {td/beta.shape[0]}')

In [ ]:
cvz.shape

In [ ]:
# note that this helper function does three different things:
# (i) plots the observed data;
# (ii) plots the predictions from the learned GP after conditioning on data;
# (iii) plots samples from the GP prior (with no conditioning on observed data)

def plot(plot_observed_data=False, plot_predictions=False, n_prior_samples=0,
         model=None, kernel=None, n_test=500):

    plt.figure(figsize=(12, 6))
    if plot_observed_data:
        plt.plot(X.numpy(), y.numpy(), 'kx')
    if plot_predictions:
        Xtest = torch.linspace(-0.5, 5.5, n_test)  # test inputs
        # compute predictive mean and variance
        with torch.no_grad():
            if type(model) == gp.models.VariationalSparseGP:
                mean, cov = model(Xtest, full_cov=True)
            else:
                mean, cov = model(Xtest, full_cov=True, noiseless=False)
        sd = cov.diag().sqrt()  # standard deviation at each input point x
        plt.plot(Xtest.numpy(), mean.numpy(), 'r', lw=2)  # plot the mean
        plt.fill_between(Xtest.numpy(),  # plot the two-sigma uncertainty about the mean
                         (mean - 2.0 * sd).numpy(),
                         (mean + 2.0 * sd).numpy(),
                         color='C0', alpha=0.3)
    if n_prior_samples > 0:  # plot samples from the GP prior
        Xtest = torch.linspace(-0.5, 5.5, n_test)  # test inputs
        noise = (model.noise if type(model) != gp.models.VariationalSparseGP
                 else model.likelihood.variance)
        cov = kernel.forward(Xtest) + noise.expand(n_test).diag()
        samples = dist.MultivariateNormal(torch.zeros(n_test), covariance_matrix=cov)\
                      .sample(sample_shape=(n_prior_samples,))
        plt.plot(Xtest.numpy(), samples.numpy().T, lw=2, alpha=0.4)

    plt.xlim(-0.5, 5.5)